In [1]:
import os

# 3번 GPU만 사용
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

/usr/anaconda3/envs/med/lib/python3.12/importlib/__init__.py:90: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


In [9]:
# 1. HuggingFace 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

In [3]:
# 2. Chroma DB 로드 (임베딩 함수와 함께 설정)
persist_directory = "/home/students/cs/sjuhun1/EEVE_rag/chroma_db_eeve"  # 이전에 저장한 DB의 경로
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [4]:
# 3. 질문 입력
user_message = "나잘스프레이오리지날이 무엇인가요?"

In [5]:
# 4. Chroma DB에서 유사한 문서 검색
retrieved_docs = vector_store.similarity_search(user_message)

In [7]:
# 5. 검색된 문서들을 하나의 텍스트로 병합
context = "\n".join([doc.page_content for doc in retrieved_docs])

print(context)

In [2]:
import transformers
import torch
import os

# GPU를 고정하기 위해 환경 변수 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

model_id = "beomi/Llama-3-Open-Ko-8B"

# 모델을 3번 GPU에 할당, torch_dtype으로 bfloat16 설정
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device= 3  # GPU 0은 CUDA_VISIBLE_DEVICES 설정으로 인해 실제로 3번 GPU가 사용됨
)

# 모델을 평가 모드로 설정
pipeline.model.eval()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
import transformers
import torch

model_id = "beomi/Llama-3-Open-Ko-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map={"", 3},  # 3번 GPU에 고정, 위에서 이미 3번 gpu로 고정해서 3번으로 잡히는 것 같음. os.environ["CUDA_VISIBLE_DEVICES"] = "3"
)

pipeline.model.eval()

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Could not load model zoohun/med_llama-3-ko-8B with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "/usr/anaconda3/envs/med/lib/python3.12/site-packages/transformers/pipelines/base.py", line 288, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/anaconda3/envs/med/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py", line 564, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/anaconda3/envs/med/lib/python3.12/site-packages/transformers/modeling_utils.py", line 4014, in from_pretrained
    ) = cls._load_pretrained_model(
        ^^^^^^^^^^^^^^

In [6]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    "zoohun/med_llama-3-ko-8B",
    quantization_config=quant_config,
    device_map="auto"
)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/anaconda3/envs/med/lib/python3.12/site-packages/transformers/quantizers/auto.py:182: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Supplied state dict for model.layers.22.mlp.down_proj.weight does not contain `bitsandbytes__*` and possibly other `quantized_stats` components.

In [18]:
PROMPT = '''
당신은 유능한 약품 처방사입니다.

사용자들의 질문에 대해 올바른 약품을 처방해주세요.

다음 문서의 내용을 바탕으로 질문에 답하세요:\n
{}
\n
'''.format(context)

print(PROMPT)


당신은 유능한 약품 처방사입니다.

사용자들의 질문에 대해 올바른 약품을 처방해주세요.

다음 문서의 내용을 바탕으로 질문에 답하세요:


약품 품목 기준 코드 : '201404778'
제품명 : '1.나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명 : Nasal Spray Original) 2.나잘스프레이엑스트라모이스쳐라이징(옥시메타졸린염산염)(수출용)(수출명 : Nasal Spray Extra Moisturizing)'
제품영문명 : 'Nasal Spray Original,  Nasal Spray Extra Moisturizing'
주성분 : '옥시메타졸린염산염'
주성분영문 : 'Oxymetazoline Hydrochloride'
첨가제 : '벤잘코늄염화물,에데트산나트륨수화물,정제수,농글리세린'
의약품 분류 기준 : '일반의약품'


약품 품목 기준 코드 : '201210198'
제품명 : '퓨어에이드나잘스프레이오리지날(옥시메타졸린염산염)(수출용)(수출명:Pure Aid Nasal Spray Original,  Pure Aid Nasal Spray Extra Moisturizing, ASSURED NASAL RELIEF SPRAY PUMP MIST ANTI-DRIP ORIGINAL Oxymetazoline HCI-Nasal Decongestant,

VM PLUS Nasal Spray, Family Care Nasal Spray Original,

Family Care Severe Congestion Nasal Spray ,

Leader Extra Moisturizing Nasal Spray Original,

Leader Severe Congestion Nasal Spray)'
제품영문명 : 'Pure Aid Nasal Spray Original'
주성분 : '옥시메타졸린염산염'
주성분영문 : 'Oxymetazoline Hydrochloride'
첨가제 : '인산이수소나트륨수화물,벤질알코올,프로필렌글리콜,dl-캄파,폴리소르베이트80,인산수소나트

In [21]:
instruction = user_message

print(instruction)

나잘스프레이오리지날이 무엇인가요?


In [22]:
messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

In [23]:
prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

In [24]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


In [25]:
outputs = pipeline(
    prompt,
    max_new_tokens=2048,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [26]:
print(outputs[0]["generated_text"][len(prompt):])

나잘스프레이오리지널(Nasal Spray Original)은 옥시메타졸린염산염(Oxymetazoline Hydrochloride)을 주성분으로 하는 일반의약품입니다. 이 약품은 코의 염증과 비침을 줄이고, 코를 막아주는 데 사용됩니다. 옥시메타졸린은 코의 혈관을 수축시키고, 염증을 줄여 코를 가벼운 상태로 유지하는 데 도움을 줍니다. 이 약품은 일반적으로 3-5일 동안 사용하는 것이 권장되며, 오래 사용하면 의존성과 반응이 줄어드는 현상이 발생할 수 있으므로 주의가 필요합니다.
